In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data.csv')
print(df.shape)
df.head()

(70943, 4)


,query_id,relevance,id,prediction
0,2,2.0,6553605,0.151981
1,2,2.0,7602182,-0.015192
2,2,2.0,9025828,-0.044992
3,2,2.0,4980745,-0.024775
4,2,2.0,2883596,-0.054759


In [36]:
df = df.sort_values(['query_id', 'prediction', 'relevance'], ascending=[False, False, True])
df.head()

,query_id,relevance,id,prediction
70940,22720,2.000000,3320579,0.274801
70941,22720,2.000000,961284,0.037761
70942,22720,0.000016,2796293,-1.038155
70938,22700,2.000000,5417722,0.263022
70937,22700,2.000000,1341365,0.119773


In [37]:
a = np.array([2, 1, 1, 3])
b = np.array([4, 5, 2, 1])
np.lexsort([a, b])

array([3, 2, 0, 1])

In [53]:
def dcg_score(y_true, y_score):
    order = np.lexsort([y_true, -y_score])
#     print(y_score, order)
#     print(y_true, order, np.take(y_true, order))
    y_true = np.take(y_true, order)
    gain = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)

def ndcg_matrix(y_true, y_score):
    N = y_true.shape[0]
#     m = np.zeros((N, N))
    dcg = dcg_score(y_true, y_score)
    m = np.diag([dcg] * N)
    for i in range(N):
        for j in range(i + 1, N):
            y_score_ = y_score.copy()
            y_score_[i] = y_score[j]
            y_score_[j] = y_score[i]
            m[i, j] = dcg_score(y_true, y_score_)
            m[j, i] = m[i, j]
    dcg_perfect = dcg_score(np.sort(y_true), np.arange(N))
    return m / dcg_perfect

In [55]:
dfs = []
for q in df.query_id.unique():
    df_ = df[df.query_id == q]
    r = df_.relevance.values
    s = df_.prediction.values
    N = r.shape[0]
    y = np.tile(r[:, None], N)
    y = np.where(y - y.T > 0, 1, 0)
    s_diff = np.tile(s[:, None], N)
    s_diff = s_diff - s_diff.T
    derivative = (1 - y) - 1 / (1 + np.exp(s_diff))
    ndcg = ndcg_matrix(r, s)
    d_ndcg = ndcg - ndcg[0, 0]
    lambda_pairwise = derivative * d_ndcg
    lambda_pairwise[~y] = -lambda_pairwise[~y]
    lambda_ = lambda_pairwise.sum(-1)
#     print(lambda_)
    df_['lambda'] = lambda_
    dfs.append(df_)

/home/ilivans/.virtualenvs/cmn36/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ilivans/.virtualenvs/cmn36/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


In [69]:
lambda_pairwise.diagonal()

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0., -0., -0.])

In [56]:
df_new = pd.concat(dfs)

In [62]:
df_new['lambda'].isna().sum()

3341

In [61]:
df_new[df_new['lambda'].isna()].head(10)

,query_id,relevance,id,prediction,lambda
70925,22638,0.0,4893405,-0.127998,NaN
70926,22638,0.0,8563423,-0.143283,NaN
70927,22638,0.0,8176203,-1.038415,NaN
70900,22528,0.0,2534037,-0.076473,NaN
70901,22528,0.0,3320475,-0.177393,NaN
70902,22528,0.0,5941917,-0.187048,NaN
70903,22528,0.0,4106910,-0.189414,NaN
70904,22528,0.0,5679776,-1.038415,NaN
70892,22496,0.0,4631171,0.183242,NaN
70891,22496,0.0,6204032,-0.000716,NaN


In [63]:
df_new.to_csv('lambda.csv', index=False)

In [66]:
subm = ' '.join(map(str, df_new.sort_values('id')['lambda'].fillna(0).values))

In [68]:
with open('submission.txt', 'w') as f:
    f.write(subm)